In [ ]:
"""
confuse
反正 metric 只接受標籤(預測完得class)
注意  init有給 name???
然後 state變數  通通用 tensor.Var ???

這個F1 scope 已經寫死  只能分兩個class

"""
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        '''initializes attributes of the class'''
        
        # call the parent class init
        super(F1Score, self).__init__(name=name, **kwargs)

        # Initialize Required variables
        # true positives
        self.tp = tf.Variable(0, dtype = 'int32')
        # false positives
        self.fp = tf.Variable(0, dtype = 'int32')
        # true negatives
        self.tn = tf.Variable(0, dtype = 'int32')
        # false negatives
        self.fn = tf.Variable(0, dtype = 'int32')
    """
    這裡重點在於 y_true, y_pred 到底長怎樣
    懂了  看起來就是1D vector 透過tf.math.confusion_matrix 轉換成2D 矩陣刑事
    tf.math.confusion_matrix 是可以根據class數量來調整conf_matrix
    
    """
    def update_state(self, y_true, y_pred, sample_weight=None):
        '''
        Accumulates statistics for the metric
        
        Args:
            y_true: target values from the test data
            y_pred: predicted values by the model
        '''

        # Calulcate confusion matrix.
        """
        注意 numclss 是從1開始算
        """
        conf_matrix = tf.math.confusion_matrix(y_true, y_pred, num_classes=2)
        
        # Update values of true positives, true negatives, false positives and false negatives from confusion matrix.
        self.tn.assign_add(conf_matrix[0][0])
        self.tp.assign_add(conf_matrix[1][1])
        self.fp.assign_add(conf_matrix[0][1])
        self.fn.assign_add(conf_matrix[1][0])

    def result(self):
        '''Computes and returns the metric value tensor.'''

        # Calculate precision
        if (self.tp + self.fp == 0):
            precision = 1.0
        else:
            precision = self.tp / (self.tp + self.fp)
      
        # Calculate recall
        if (self.tp + self.fn == 0):
            recall = 1.0
        else:
            recall = self.tp / (self.tp + self.fn)

        # Return F1 Score
        ### START CODE HERE ###
        f1_score =  2 * ((precision * recall) / (precision + recall))
        ### END CODE HERE ###
        print("f1_score : {}".format(f1_score))
        return f1_score

    def reset_states(self):
        '''Resets all of the metric state variables.'''
        
        # The state of the metric will be reset at the start of each epoch.
        self.tp.assign(0)
        self.tn.assign(0) 
        self.fp.assign(0)
        self.fn.assign(0)

In [ ]:
"""
基本上 metric 就是4個變數 TP TN FP FN與其各種計算
"""
class MyMetric(tf.keras.metrics.Metric):
    def __init__(self, name='MyMetric', **kwargs):
        '''initializes attributes of the class'''
        
        # call the parent class init
        super(MyMetric, self).__init__(name=name, **kwargs)

        # Initialize Required variables
        # true positives
        self.tp = tf.Variable(0, dtype = 'int32')
        # false positives
        self.fp = tf.Variable(0, dtype = 'int32')
        # true negatives
        self.tn = tf.Variable(0, dtype = 'int32')
        # false negatives
        self.fn = tf.Variable(0, dtype = 'int32')
    def update_state(self, y_true, y_pred, sample_weight=None):
        '''
        Accumulates statistics for the metric
        
        Args:
            y_true: target values from the test data
            y_pred: predicted values by the model
        '''

        # Calulcate confusion matrix.
        """
        注意 numclss 是從1開始算  不同的clas 樹木 conf_matrix的取直方式會不一樣
        """
        conf_matrix = tf.math.confusion_matrix(y_true, y_pred, num_classes=2)
        print("in MyMetric conf_matrix :{}".format(conf_matrix))
        # Update values of true positives, true negatives, false positives and false negatives from confusion matrix.
        self.tn.assign_add(conf_matrix[0][0])
        self.tp.assign_add(conf_matrix[1][1])
        self.fp.assign_add(conf_matrix[0][1])
        self.fn.assign_add(conf_matrix[1][0])

    def result(self):
        '''Computes and returns the metric value tensor.'''
        ACC = (self.tp + self.tn) /(self.tp + self.tn + self.fp + self.fn)   #預測隊的 / 全部
        Specificity = self.tn / (self.tn + self.fp)  #跟 Recall 依樣  差別在於是針對錯誤的正確率
        print("ACC : {}, Precisiion: {},Recall : {},  Specificity: {}".format(ACC,Precisiion, Recall, Specificity))
        return (ACC, Precisiion, Recall, Specificity)

    def reset_states(self):
        '''Resets all of the metric state variables.'''
        
        # The state of the metric will be reset at the start of each epoch.
        self.tp.assign(0)
        self.tn.assign(0) 
        self.fp.assign(0)
        self.fn.assign(0)

In [ ]:
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau

In [ ]:
class VisCallback(tf.keras.callbacks.Callback):
    def __init__(self, inputs, ground_truth, display_freq=10, n_samples=10):
        self.inputs = inputs
        self.ground_truth = ground_truth
        self.images = []
        self.display_freq = display_freq
        self.n_samples = n_samples

    def on_epoch_end(self, epoch, logs=None):
        # Randomly sample data
        indexes = np.random.choice(len(self.inputs), size=self.n_samples)
        X_test, y_test = self.inputs[indexes], self.ground_truth[indexes]
        predictions = np.argmax(self.model.predict(X_test), axis=1)

        # Plot the digits
        display_digits(X_test, predictions, y_test, epoch, n=self.display_freq)

        # Save the figure
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        image = Image.open(buf)
        self.images.append(np.array(image))

        # Display the digits every 'display_freq' number of epochs
        if epoch % self.display_freq == 0:
            plt.show()

    def on_train_end(self, logs=None):
        imageio.mimsave(GIF_PATH, self.images, fps=1)

In [ ]:
class MycustomerCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold = 0.7):
        super(MycustomerCallback, self).__init__()
        self.threshold = threshold
    def on_epoh_end(self, epoch, logs =None):
        print("logs = {}".format(logs))
        ratio = logs["val_loss"] / logs["loss"]
        print("Epoch: {}, Val/Train loss ratio: {:.2f}".format(epoch, ratio))

        if ratio > self.threshold:
            print("Stopping training...")
            self.model.stop_training = True
#     def on_train_batch_end(self, batch, logs=None):
#         print("MycustomerCallback , on_train_barch_end epoh:{}, logs = {}".format(batch, logs))

In [ ]:
%load_ext tensorboard
%tensorboard --logdir log_dir

In [ ]:
"""
#BCE_sigmoid_Metrics
用於 sigmoid + BCE 裡面包含多個2分類 metric


"""

class BCE_sigmoid_Metrics(tf.keras.metrics.Metric):

    def __init__(self, labels, from_logits,  name='f1_score', **kwargs):
        '''initializes attributes of the class'''
        
        # call the parent class init
        super(BCE_sigmoid_Metrics, self).__init__(name=name, **kwargs)
        self.labels = labels
        self.from_logits = from_logits
        self.num_class = len(self.labels)
        self.BceMetrics = []
        for _ in range(self.num_class):
            self.BceMetrics.append(MyConfuse_Metric(from_logits = True))
    """
    makeCms return shape = (b,nclass)
    describe:
        依序製作cm 並使用cm更新自己的metrics
    Args:
        都要是1D vector
        y_true: target values from the test data
        y_pred: predicted values by the model
    注意:
        tf.math.confusion_matrix 接收的也不是onehot  ex:  [1, 2, 4], [2, 2, 4] 而且是 label not logits
        update confuse matric 不可以使用logits 要轉乘標籤
    """
    def update_state(self, y_true, y_pred, sample_weight=None):
        def makeCms(y_true, y_pred):
            allcms = []
            assert y_true.shape == y_pred.shape , "ytrue.shape != ypred.shape"
            batch_size = y_true.shape[0]
            class_num = y_true.shape[1]
            onecolcoms = []
            for i, j in itertools.product(range(batch_size), range(class_num)):
                temp = np.zeros(shape = (2,2),dtype = np.int32)
                temp[int(not y_true[i][j].numpy())][int(not y_pred[i][j].numpy())] = 1
                onecolcoms.append(temp)
                if j == class_num-1:
                    allcms.append(onecolcoms)
                    onecolcoms = []
            return allcms

        y_true = tf.cast(tf.round(y_true),"int64")
        y_pred = tf.cast(tf.round(y_pred),"int64")
        
        batchCms = makeCms(y_true, y_pred)
        # Update values of true positives, true negatives, false positives and false negatives from confusion matrix.
        for oneBatchCm in batchCms:
            for metric, cm in zip(self.BceMetrics, oneBatchCm):
                metric.update_state_byCm(cm)

    def showCMs(self):
        for metric,label in zip(self.BceMetrics, self.labels):
            metric.plot_confusion_matrix(label)
            plt.show()
                
    def result(self):
        result = []
        for metric, label in zip(self.BceMetrics, self.labels):
            score = metric.result_by_collections(label)
            result.append(score[3])
        return tf.reduce_mean(result)
    
    def result_pd(self):
        """
        每個metric return  pd(acc, pre, recall ,f1) 
        """
        result = []
        for metric, label in zip(self.BceMetrics, self.labels):
            result.append(metric.result_pd(label))
        return result
    def reset_states(self):
        for metric in self.BceMetrics:
            metric.reset_states()

In [ ]:
"""
#MyConfuse_Metric
Precisiion = self.tp / (self.tp + self.fp)    #只看預測隊的部分  也就是對的正確機率
Recall = self.tp /(self.tp + self.fn)   #TP + FN就是所有資料裡面判斷是P的部分  也就是在全部P裡面到底預測對了多少個
只能記錄一個2分類結果
"""

class MyConfuse_Metric(tf.keras.metrics.Metric):

    def __init__(self, from_logits, name='f1_score', **kwargs):
        '''initializes attributes of the class'''
        
        # call the parent class init
        super(MyConfuse_Metric, self).__init__(name=name, **kwargs)
        self.from_logits = from_logits
        # Initialize Required variables
        # true positives
        self.tp = tf.Variable(0, dtype = 'int32')
        # false positives
        self.fp = tf.Variable(0, dtype = 'int32')
        # true negatives
        self.tn = tf.Variable(0, dtype = 'int32')
        # false negatives
        self.fn = tf.Variable(0, dtype = 'int32')

    def update_state(self, y_true, y_pred, sample_weight=None):

        '''
        Accumulates statistics for the metric
        
        
        Args:
            都要是1D vector
            y_true: target values from the test data
            y_pred: predicted values by the model
        '''
        
        """
        if is muti label,class 
            必須要先處理 y_true, y_pred 維度問題

        """
        # Calulcate confusion matrix.
        """
        labels	1-D Tensor of real labels for the classification task.
        predictions	1-D Tensor of predictions for a given classification.
        tf.math.confusion_matrix 接收的也不是onehot  ex:  [1, 2, 4], [2, 2, 4] 而且是 label not logits
        
        必須製作出cm 才有辦法存到tp,tn....
        """
        if self.from_logits:
            y_true = tf.cast(tf.round(y_true),"int64")
            y_pred = tf.cast(tf.round(y_true),"int64")
        conf_matrix = tf.math.confusion_matrix(y_true, y_pred, num_classes=2)
        
        # Update values of true positives, true negatives, false positives and false negatives from confusion matrix.
        self.tn.assign_add(conf_matrix[0][0])
        self.tp.assign_add(conf_matrix[1][1])
        self.fp.assign_add(conf_matrix[0][1])
        self.fn.assign_add(conf_matrix[1][0])
        
    def update_state_byCm(self, conf_matrix):
#         print(f"in update_state_byCm conf_matrix:{conf_matrix} type : {type(conf_matrix)}")
        self.tn.assign_add(conf_matrix[0][0])
        self.tp.assign_add(conf_matrix[1][1])
        self.fp.assign_add(conf_matrix[0][1])
        self.fn.assign_add(conf_matrix[1][0])
    
    """
    使用在 自訂metric 裡面  使用tp,tn....
    """
    def plot_confusion_matrix(self, title, label_names = ["pos","neg"], cmap=plt.cm.Blues):
        cm = np.array([[self.tn,self.fp],[self.fn, self.tp]])
        fmt = 'd'
        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, format(cm[i, j], fmt),
            horizontalalignment="center",
            color="black" if cm[i, j] < thresh else "white")
        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        tick_marks = np.arange(len(label_names))
        plt.xticks(tick_marks, label_names, rotation=45)
        plt.yticks(tick_marks, label_names)
        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        

    def result(self):
        '''Computes and returns the metric value tensor.'''

        # Calculate precision
        if (self.tp + self.fp == 0):
            precision = 1.0
        else:
            precision = self.tp / (self.tp + self.fp)
      
        # Calculate recall
        if (self.tp + self.fn == 0):
            recall = 1.0
        else:
            recall = self.tp / (self.tp + self.fn)
        
        ACC = (self.tp + self.tn) /(self.tp + self.tn + self.fp + self.fn)

        # Return F1 Score
        ### START CODE HERE ###
        f1_score =  2 * ((precision * recall) / (precision + recall))
        ### END CODE HERE ###
        print(f"Acc : {ACC}, precision : {precision}, recall : {recall}, f1_score : {f1_score}")
        return f1_score
    
    def result_pd(self,index):
        import pandas as pd
        if (self.tp + self.fp == 0):
            precision = 1.0
        else:
            precision = self.tp / (self.tp + self.fp)
      
        # Calculate recall
        if (self.tp + self.fn == 0):
            recall = 1.0
        else:
            recall = self.tp / (self.tp + self.fn)
        
        ACC = (self.tp + self.tn) /(self.tp + self.tn + self.fp + self.fn)

        # Return F1 Score
        ### START CODE HERE ###
        f1_score =  2 * ((precision * recall) / (precision + recall))
        
        result = pd.DataFrame([[ACC,precision,recall,f1_score]], columns = ["ACC","precision","recall","f1_score"], index = index)
        return result
    
    def result_by_collections(self,label_name):
        # Calculate precision
        if (self.tp + self.fp == 0):
            precision = 1.0
        else:
            precision = self.tp / (self.tp + self.fp)
      
        # Calculate recall
        if (self.tp + self.fn == 0):
            recall = 1.0
        else:
            recall = self.tp / (self.tp + self.fn)
        
        ACC = (self.tp + self.tn) /(self.tp + self.tn + self.fp + self.fn)

        # Return F1 Score
        ### START CODE HERE ###
        f1_score =  2 * ((precision * recall) / (precision + recall))
        ### END CODE HERE ###
        print(f"label_name: {label_name}, Acc : {ACC}, precision : {precision}, recall : {recall}, f1_score : {f1_score}")
        return (ACC,precision,recall,f1_score)
    
    def reset_states(self):
        '''Resets all of the metric state variables.'''
        
        # The state of the metric will be reset at the start of each epoch.
        self.tp.assign(0)
        self.tn.assign(0) 
        self.fp.assign(0)
        self.fn.assign(0)
